# Double Payment Analysis


Question: Are there any double payments?

What are double payments?

1. Paid a declaration at least twice.
2. Paid prepaid travel costs again.

__Idea__: declaration number should be unique and only paid once.

__Problem__: Multiple declaration might be handled multiple times if not the the complete amount was paid.

In [1]:
import sys
sys.path.insert(1, '../')

In [23]:
from src.io import DOM_DEC, INT_DEC, PER, PRE, read_log, to_dataframe                 

# util
from matplotlib import pyplot as plot
from datetime import datetime, timedelta
import numpy as np
from collections import Counter
import pandas as pd
from functools import reduce

In [4]:
def check_for_double_payments(log):
    declarations = []
    double_booked = []

    for trace in log:
        for event in trace:
            if event['concept:name'] == 'Payment Handled':
                dec = trace.attributes['concept:name']
                try:
                    dec = int(dec.split('declaration')[1].strip())
                except:
                    # print(trace.attributes['concept:name'] +': ' +dec)
                    pass
                if dec in declarations:
                    double_booked.append(dec)
                else:
                    declarations.append(dec)
    
    print('There are %s double payments.' %len(set(double_booked)))
    return double_booked

## Domestic Declarations

In [5]:
log_dom_dec = read_log(DOM_DEC)

parsing log, completed traces :: 100%|██████████| 10357/10357 [00:04<00:00, 2106.56it/s]


In [6]:
double_payments = check_for_double_payments(log_dom_dec)

There are 0 double payments.


## International Declarations

In [7]:
log_int_dec = read_log(INT_DEC)

parsing log, completed traces :: 100%|██████████| 6323/6323 [00:14<00:00, 444.81it/s]


In [8]:
double_payments = check_for_double_payments(log_int_dec)

There are 0 double payments.


## Travel Permits

### Search for double payments

In [9]:
log_per = read_log(PER)

parsing log, completed traces :: 100%|██████████| 6426/6426 [00:20<00:00, 321.11it/s]


__Question:__
Relation between: Requested amount, requested budget, and total declared?

In [11]:
double_payments = check_for_double_payments(log_per)

There are 1202 double payments.


Examples of cases with multiple payments:

In [12]:
double_booked = list(set(double_payments))
print(double_booked[:3])

['travelpermit44505', 'travelpermit54681', 'travelpermit29696']


### Investigate Permits with multiple payments

Create new subset of the log, containing only cases with multiple 'payment handled' activities.

In [13]:
subset = list(filter(lambda trace: trace.attributes['concept:name'] in double_booked , log_per))

In [16]:
def calc_potential_loss(amounts: list):
    loss = 0
    k = list(Counter(amounts).keys())
    v = list(Counter(amounts).values())
    for idx, value in enumerate(v):
        if value > 1:
            loss += k[idx] * (value - 1) 
    return loss

def amount_to_int(value: str):
    if (',' in value):
        value = value.replace(',', '')
        return int(value)
    elif ('.' in value):
        return float(value)
    elif value == '':
        value = 0
    
    return int(value)

In [19]:
suspicious_cases = []
trace_length = []
number_of_payments = []
potential_loss = []

for trace in subset:
    # get all requested amounts
    requested_amounts_ids = list(filter(lambda attr: 'RequestedAmount' in attr, trace[0]))
    
    amounts = []
    
    for request_id in requested_amounts_ids:
        amounts.append(amount_to_int(str(trace[0][request_id])))
    
    # if multiple time the same amount is requested
    if len(amounts) is not len(set(amounts)):
        
        loss = calc_potential_loss(amounts)
        if loss > 0:
            suspicious_cases.append(trace.attributes['concept:name'].replace('travelpermit', ''))
            trace_length.append(len(trace))
            number_of_payments.append(len(amounts))
            potential_loss.append(loss)
        
        # print('Potential fraud in case: %s (%s events, loss: %s)' %(trace.attributes['id'], len(trace), loss))

fraud = {'Trace Length' : trace_length, '# of Payments' : number_of_payments, 'Potential Loss' : potential_loss}
        
results = pd.DataFrame(fraud, 
                       columns = ['Trace Length','# of Payments', 'Potential Loss'], 
                       index=suspicious_cases)
results = results.sort_values(by='Potential Loss', ascending=False)

print(results)
print('-------------------------------------------------')
print('Total number of suspicious cases: %s' %len(suspicious_cases))

Trace Length  # of Payments  Potential Loss
54518            24              3   475704.945921
72935            23              3     2726.802196
58250            24              3     1397.137205
83423            14              3     1394.160631
78577            19              3      667.064113
47318            44              6      426.550502
49668            33              4      375.555258
32177            33              5      296.329435
80539            33              6      231.014366
21585            21              4      227.631376
32577            61             10      226.000000
45618            53              9      208.000000
64095            45              5      160.000000
66595            37              6      150.000000
14193            30              3       88.000000
1726             15              2       57.000000
22776            81             15       43.718358
62668            39              5       35.000000
81226            25              5    

### Compare different attributes

- (case)_RequestedAmount_X
- (case)_Overspent
- (case)_OverspentAmount
- (case)_RequestedBudget
- (case)_TotalDeclared

In [20]:
log_per[0][0]

{'lifecycle:transition': 'complete', 'time:timestamp': datetime.datetime(2017, 1, 9, 0, 0, tzinfo=datetime.timezone(datetime.timedelta(seconds=3600))), '(case)_ActivityNumber': 'UNKNOWN', '(case)_BudgetNumber': 'budget 2233', '(case)_DeclarationNumber_0': 'declaration number 72591', '(case)_OrganizationalEntity': 'organizational unit 65455', '(case)_Overspent': 'false', '(case)_OverspentAmount': '-203.9666139130759', '(case)_ProjectNumber': 'UNKNOWN', '(case)_RequestedAmount_0': '20', '(case)_RequestedBudget': '245.0601031890994', '(case)_TaskNumber': 'UNKNOWN', '(case)_TotalDeclared': '14.569043850933994', '(case)_dec_id_0': 'declaration 72590', '(case)_id': 'travel permit 72588', '(case)_travel_permit_number': 'travel permit number 72589', 'concept:name': 'Start trip', 'id': 'rv_travel permit 72588_6', 'org:resource': 'STAFF MEMBER', 'org:role': 'EMPLOYEE'}

#### Relation of TotalDeclared and RequestedAmounts

 $Total Declared = \sum_{i=0}^{n} RequestedAmount_i$

In [21]:
def convert_amount(amount:str):
    if ',' in amount:
        amount = amount.replace(',', '')
    return float(amount)

In [24]:
trace = log_per[0]

suspicious_cases = []
for trace in log_per:
    td = float(trace[0]['(case)_TotalDeclared'])
    requests = list(filter(lambda x: '(case)_RequestedAmount' in x, trace[0]))
    amounts = list(map(lambda x: convert_amount(trace[0][x]), requests))
    if len(amounts) > 0:
        total_amounts = reduce(lambda x,y: x + y, amounts)
    else:
        # no amount requested
        total_amounts = 0
    
    if total_amounts < td:
        suspicious_cases.append((trace.attributes['concept:name'], total_amounts, td))

Inspect one example:

In [25]:
suspicious_cases[0]

('travelpermit76036', 587.0, 635.2514478871398)

In [26]:
cases = len(log_per)
p = len(suspicious_cases) / cases * 100

In [28]:
print('In %s cases (%0.2f perc.) the total declared value was higher, than the sum of the requested amounts.' %(len(suspicious_cases), p))

In 2625 cases (40.85 perc.) the total declared value was higher, than the sum of the requested amounts.


What is the average difference between those numbers?

In [31]:
diff = []
for case in suspicious_cases:
    diff.append(case[2] - case[1])

avg = np.mean(diff)
print('avg: %0.2f, max: %0.2f, min %0.2f' %(avg, max(diff), min(diff)))

avg: 204.95, max: 1747.76, min 0.12


#### Compare overspent and overspent amount

In [32]:
def convert_bool(value:str):
    if 'false' in value:
        return False
    elif 'true' in value:
        return True

In [33]:
suspicious_cases = []

for trace in log_per[0:1]:
    overspent = convert_bool(trace[0]['(case)_Overspent'])
    overspent_amount = convert_amount(trace[0]['(case)_OverspentAmount'])
    if not overspent and overspent_amount <= 0:
        pass
    else:
        suspicious_cases.append(trace.attributes['concept:name'])

In [35]:
print('There are %s cases with a missmatch of overspent and overspent amount.' %len(suspicious_cases))

There are 0 cases with a missmatch of overspent and overspent amount.


### Compare Requested Budget, Total Declared, total amout and overspent

In [36]:
suspicious_cases = []

for trace in log_per:
    requested_budget = convert_amount(trace[0]['(case)_RequestedBudget'])
    td = float(trace[0]['(case)_TotalDeclared'])
    overspent = convert_bool(trace[0]['(case)_Overspent'])
    requests = list(filter(lambda x: '(case)_RequestedAmount' in x, trace[0]))
    amounts = list(map(lambda x: convert_amount(trace[0][x]), requests))
    if len(amounts) > 0:
        total_amounts = reduce(lambda x,y: x + y, amounts)
    else:
        # no amount requested
        total_amounts = 0
    
    if (td > requested_budget and total_amounts > requested_budget) and not overspent:
        suspicious_cases.append((trace.attributes['concept:name'], td, requested_budget, total_amounts, overspent))
        

In [37]:
not_overspent = 0

for trace in log_per:
    if not convert_bool(trace[0]['(case)_Overspent']):
        not_overspent += 1    

p = len(suspicious_cases) / not_overspent * 100 

In [39]:
print('There are %s cases (%0.2f perc. of not overspent cases) where the total declared value and the sum of requested amounts is higher, the the requested budget but still not marked as overspent' %(len(suspicious_cases), p))

There are 594 cases (12.98 perc. of not overspent cases) where the total declared value and the sum of requested amounts is higher, the the requested budget but still not marked as overspent


In [40]:
suspicious_cases[2]

('travelpermit77372', 943.2928285279097, 611.9260247840641, 700.0, False)